<a href="https://colab.research.google.com/github/Gotothenewsky/AI-Project/blob/main/%5BProject%202%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 버전

!pip install category_encoders        ## 인코더 설치
!pip install pandas-profiling==3.1.0  ## 판다스 프로파일링 설치

In [3]:
#주요 라이브러리 

import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 사이킷런 - 데이터 셋 나누기
from sklearn.model_selection import train_test_split

# 사이킷런 - encoders & imputers
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer, KNNImputer

# 사이킷런 - 파이프라인
from sklearn.pipeline import make_pipeline

# 사이킷런 - 모델 평가 지표
from sklearn.metrics import accuracy_score, mean_absolute_error, f1_score

# CV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, validation_curve


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
# 자동차 관련 데이터셋 출력

test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

target = 'Price'

In [8]:
train.dtypes

ID                    int64
Price                 int64
Levy                 object
Manufacturer         object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume        object
Mileage              object
Cylinders           float64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
dtype: object

In [9]:
test.dtypes

ID                    int64
Levy                 object
Manufacturer         object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume        object
Mileage              object
Cylinders             int64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
Price               float64
dtype: object

테스트, 훈련 데이터 순서는 다르지만 동일한 데이터 타입가짐

In [10]:
train[target].value_counts(normalize=True)

15681    0.014555
470      0.014243
14113    0.012684
392      0.012580
314      0.012216
           ...   
42601    0.000052
149      0.000052
54349    0.000052
54954    0.000052
22075    0.000052
Name: Price, Length: 2315, dtype: float64

In [11]:
train.T.duplicated()

ID                  False
Price               False
Levy                False
Manufacturer        False
Model               False
Prod. year          False
Category            False
Leather interior    False
Fuel type           False
Engine volume       False
Mileage             False
Cylinders           False
Gear box type       False
Drive wheels        False
Doors               False
Wheel               False
Color               False
Airbags             False
dtype: bool

In [13]:
train.describe(exclude='number')

,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Gear box type,Drive wheels,Doors,Wheel,Color
count,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237,19237
unique,559,65,1590,11,2,7,107,7687,4,3,3,2,16
top,-,HYUNDAI,Prius,Sedan,Yes,Petrol,2,0 km,Automatic,Front,04-May,Left wheel,Black
freq,5819,3769,1083,8736,13954,10150,3916,721,13514,12874,18332,17753,5033


In [14]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
Leather interior,19237,2,Yes,13954
Wheel,19237,2,Left wheel,17753
Drive wheels,19237,3,Front,12874
Doors,19237,3,04-May,18332
Gear box type,19237,4,Automatic,13514
Fuel type,19237,7,Petrol,10150
Category,19237,11,Sedan,8736
Color,19237,16,Black,5033
Manufacturer,19237,65,HYUNDAI,3769
Engine volume,19237,107,2,3916


In [18]:
train['Prod. year'].value_counts() ## 특성의 범주 확인

2012    2155
2014    2124
2013    1963
2011    1612
2015    1549
2010    1483
2016    1476
2017     959
2008     737
2009     601
2018     500
2007     464
2005     402
2003     367
2004     364
2006     317
2019     306
2002     296
2000     279
2001     254
1998     213
1999     207
1997     151
1996     114
1995     105
2020      47
1994      42
1992      30
1993      23
1990      18
1988      12
1991      10
1986       6
1989       6
1987       5
1984       5
1985       5
1953       4
1983       3
1939       3
1978       2
1980       2
1965       2
1977       2
1974       2
1964       2
1943       1
1976       1
1957       1
1968       1
1947       1
1982       1
1981       1
1973       1
Name: Prod. year, dtype: int64